In [3]:
import pandas as pd
import re
import nltk
import sklearn
from os import path

import context as c

In [4]:
answers_path = path.join(c.DATA_PATH, 'dialog_talk_agent.xlsx')

In [5]:
df = pd.read_excel(answers_path)
df.head(10)

,Context,Text Response
0,Tell me about your personality,Just think of me as the ace up your sleeve.
1,I want to know you better,I can help you work smarter instead of harder
2,Define yourself,NaN
3,Describe yourself,NaN
4,tell me about yourself,NaN
5,all about you,NaN
6,tell me some stuff about you,NaN
7,talk some stuff about you,NaN
8,talk about yourself,NaN
9,about yourself,NaN


In [6]:
# заполняем пустые ячейки предыдущим значением
df.ffill(axis=0, inplace=True)
df.head(10)

,Context,Text Response
0,Tell me about your personality,Just think of me as the ace up your sleeve.
1,I want to know you better,I can help you work smarter instead of harder
2,Define yourself,I can help you work smarter instead of harder
3,Describe yourself,I can help you work smarter instead of harder
4,tell me about yourself,I can help you work smarter instead of harder
5,all about you,I can help you work smarter instead of harder
6,tell me some stuff about you,I can help you work smarter instead of harder
7,talk some stuff about you,I can help you work smarter instead of harder
8,talk about yourself,I can help you work smarter instead of harder
9,about yourself,I can help you work smarter instead of harder


In [7]:
def step1(text):
    for word in text:
        p = str(word).lower()
        b = re.sub(r'[^a-z0-9]', ' ', p)
        print(b)

df1 = df.head(15)
step1(df1['Context'])

tell me about your personality
i want to know you better
define yourself
describe yourself
tell me about yourself
all about you
tell me some stuff about you
talk some stuff about you
talk about yourself
about yourself
who are you
introduce yourself
i want to know more about you
what are you
what is your personality


In [8]:
s = 'tell me about your personality'

In [9]:
def convert_ps(ps):
    if ps in ['NN', 'NNS', 'NNP']:
        return 'n'
    elif ps in ['JJ', 'JJR', 'JJS']:
        return 'a'
    elif ps in ['VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
        return 'v'
    elif ps in ['RB', 'RBR', 'RBS']:
        return 'r'
    else:
        return 'n'

In [10]:
def text_normalize(text):
    words = str(text).lower()
    words = re.sub(r'^[a-z0-9]', ' ', words)
    tokens = nltk.word_tokenize(words)
    tags = nltk.pos_tag(tokens)
    lem_list = []
    lemm = nltk.WordNetLemmatizer()

    for word, tag in tags:
        pos_val = convert_ps(tag)
        lemm_token = lemm.lemmatize(word, pos_val)
        lem_list.append(lemm_token)

    return ' '.join(lem_list)

In [11]:
print(s, text_normalize(s))

tell me about your personality ell me about your personality


In [12]:
df['Lemmatize'] = df['Context'].apply(text_normalize)
df.head(10)

,Context,Text Response,Lemmatize
0,Tell me about your personality,Just think of me as the ace up your sleeve.,ell me about your personality
1,I want to know you better,I can help you work smarter instead of harder,want to know you good
2,Define yourself,I can help you work smarter instead of harder,efine yourself
3,Describe yourself,I can help you work smarter instead of harder,escribe yourself
4,tell me about yourself,I can help you work smarter instead of harder,ell me about yourself
5,all about you,I can help you work smarter instead of harder,ll about you
6,tell me some stuff about you,I can help you work smarter instead of harder,ell me some stuff about you
7,talk some stuff about you,I can help you work smarter instead of harder,alk some stuff about you
8,talk about yourself,I can help you work smarter instead of harder,alk about yourself
9,about yourself,I can help you work smarter instead of harder,bout yourself


In [13]:
stop = nltk.corpus.stopwords.words('english')

In [14]:
cv = sklearn.feature_extraction.text.CountVectorizer()
X = cv.fit_transform(df['Lemmatize']).toarray()
features = cv.get_feature_names()
df_bow = pd.DataFrame(X, columns=features)
df_bow.head(10)

c:\Users\BinaryCat17\.conda\envs\ml\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,21,about,absolutely,actually,ad,adore,advice,advise,afraid,again,...,xcuse,ya,ye,year,yes,yet,you,your,yours,yourself
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
6,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
7,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
8,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [15]:
Quest = 'Will you help me and tell me about yourself more'
tokens = nltk.word_tokenize(Quest)
tokens = [t for t in tokens if t not in stop]
Q = ' '.join(tokens)

In [16]:
Q = text_normalize(Q)
Q_bow = cv.transform([Q]).toarray()
Q_bow

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [17]:
cosine_value = 1 - sklearn.metrics.pairwise_distances(df_bow, Q_bow, metric='cosine')
df['Simil'] = cosine_value
df.head(10)

,Context,Text Response,Lemmatize,Simil
0,Tell me about your personality,Just think of me as the ace up your sleeve.,ell me about your personality,0.0
1,I want to know you better,I can help you work smarter instead of harder,want to know you good,0.0
2,Define yourself,I can help you work smarter instead of harder,efine yourself,0.0
3,Describe yourself,I can help you work smarter instead of harder,escribe yourself,0.0
4,tell me about yourself,I can help you work smarter instead of harder,ell me about yourself,0.0
5,all about you,I can help you work smarter instead of harder,ll about you,0.0
6,tell me some stuff about you,I can help you work smarter instead of harder,ell me some stuff about you,0.0
7,talk some stuff about you,I can help you work smarter instead of harder,alk some stuff about you,0.0
8,talk about yourself,I can help you work smarter instead of harder,alk about yourself,0.0
9,about yourself,I can help you work smarter instead of harder,bout yourself,0.0


In [18]:
ans_ind = cosine_value.argmax()
ans = df['Text Response'].loc[ans_ind]
print('Запрос\n', Quest, '\n', 'Ответ\n', ans)

Запрос
 Will you help me and tell me about yourself more 
 Ответ
 I'm glad to help. What can I do for you?
